## Build Data Mart
#### Contributors:
##### Tean 8: Anthony Ung, Sean Jerzewski, Gideon Kipkorir
##### Team 9: Rohith, Sneha Dasarla
##### Team 10: Anmol Brahmbhatt, Nikita Brahmbhatt, Satya

## 0. Dependencies

In [1]:
import os
from enum import Enum

#

## 1. Gather the file paths
  
  
## IMPORTANT: 
#### Most of these files are untracked on GitHub. it is each team members'   
####   &emsp; &emsp; It is each team members' individual responsibilities  
####   &emsp; &emsp; to build the Database and CSV files for themselves using the other Jupyter notebooks. 

In [2]:
FILE_PATHS = {
    'DB_TEAM_8' : '../0_SD_Team_8/store_team_8.db',
    'DB_TEAM_9' : '../0_SD_Team_9/grocery_store.db',
    'DB_TEAM_10' : '../0_SD_Team_10/grocery_team_10_v2.db',
    'PRODUCTS_CSV' : '../2_Product_Mapping/PRODUCTS_MAPPED.csv'
}

DATA_MART_NAME = './Region_C_Data_Mart.db'

In [3]:
ALL_FILES_OK = True

for file_key in FILE_PATHS:
    file_name = FILE_PATHS[file_key]
    file_exists = os.path.isfile(file_name)
    
    if(file_exists):
        print(f'OK - {file_key} - \'{file_name}\'')
    else:
        ALL_FILES_OK = False
        print(f'MISSING - {file_key} - \'{file_name}\'')

if not ALL_FILES_OK:
    raise SystemExit('\n' "ERROR!" '\n' "You are missing files!" '\n' "Read and Follow the Cell instructions provided.")

OK - DB_TEAM_8 - '../0_SD_Team_8/store_team_8.db'
OK - DB_TEAM_9 - '../0_SD_Team_9/grocery_store.db'
OK - DB_TEAM_10 - '../0_SD_Team_10/grocery_team_10_v2.db'
OK - PRODUCTS_CSV - '../2_Product_Mapping/PRODUCTS_MAPPED.csv'


#

## 2. Compile the table definitions

In [ ]:
'''
    TABLE_DEFINITIONS is a dict as follows:
        Key - the name of the table in the database
        Value - the CREATE TABLE statement for the table
    I wrote a lot of unused table definitions that will be useful
        in a later HW.
'''
TABLE_DEFINITIONS = {
    'date' : \
            'CREATE TABLE date(' \
                    'DateKey INT, ' \
                    'PrettyDate TEXT, ' \
                    'DayNumberInMonth INT, ' \
                    'DayNumberInYear INT, ' \
                    'WeekNumberInYear INT, ' \
                    'MonthNum INT, ' \
                    'MonthTxt TEXT, ' \
                    'Quarter INT, ' \
                    'Year INT,' \
                    'FiscalYear INT, ' \
                    'isHoliday INT, ' \
                    'isWeekend INT, ' \
                    'Season TEXT' ')',

    'products': \
            'CREATE TABLE products(' \
                    'ProductKey INT,' \
                    'sku INT,' \
                    'product_name TEXT, ' \
                    'product_class_id INT, ' \
                    'subcategory TEXT, ' \
                    'category TEXT, ' \
                    'department TEXT, ' \
                    'product_family TEXT, ' \
                    'size TEXT, ' \
                    'case_count INT, ' \  
                    'BrandName TEXT, ' \
                    'Manufacturer TEXT, ' \
                    'Supplier TEXT, ' \
                    'CostToStore REAL)',

    'store' : \
            'CREATE TABLE store(' \
                    'StoreKey INT, ' \
                    'StoreManager TEXT, ' \
                    'StoreStreetAddr TEXT, ' \
                    'StoreTown TEXT, ' \
                    'StoreZipCode TEXT, ' \
                    'StorePhoneNumber TEXT, ' \
                    'StoreState TEXT' ')',
    
    'sales_transactions': \
            'CREATE TABLE sales_transactions(' \
                    'DateKey INT, ' \
                    'DailyCustomerNumber INT, ' \
                    'ProductKey INT, ' \
                    'StoreKey INT, ' \
                    'QuantitySold INT, ' \
                    'TotalDollarSales REAL, ' \
                    'TotalCostToStore REAL, ' \
                    'GrossProfit REAL)',

    'sales_daily': \
            'CREATE TABLE sales_transactions(' \
                    'DateKey INT, ' \
                    'DailyCustomerNumber INT, ' \
                    'ProductKey INT, ' \
                    'StoreKey INT, ' \
                    'QuantitySoldToday INT, ' \
                    'CostOfItemsSold REAL, ' \
                    'SalesTotal REAL, ' \
                    'GrossProfit REAL)',

    'inventory_daily' : \
            'CREATE TABLE inventory_daily(' \
                    'DateKey INT, ' \
                    'ProductKey INT, ' \
                    'StoreKey INT, ' \
                    'NumAvailable INT, '
                    'CostToStoreItem FLOAT, ' \
                    'CostToStore FLOAT, ' \
                    'NumCasesPurchasedToDate INT)', 

    'inventory_quarterly' : \
            'CREATE TABLE inventory_quarterly(' \
                    'ProductKey INT, ' \
                    'StoreKey INT, ' \
                    'Quarter INT, ' \
                    'Year INT, ' \
                    'CasesPurchasedToDate INT, ' \
                    'CasesPurchasedThisQuarter INT, ' \
                    'CasesOnHand INT, ' \
                    'TotalCostToStoreThisQuarter FLOAT, ' \
                    'TotalSoldByStoreThisQuarter FLOAT, ' \
                    'TotalCostToStoreThisYTD FLOAT, ' \
                    'TotalSoldByStoreThisYTD FLOAT)'
}


#

## 3. Initialize the Database File and the Database API
#### Note: The first cell in this block is destructive.
#### If you need to see multiple versions of the database side-by-side, rename the db file before rerunning this notebook.

In [4]:
if os.path.isfile(DATA_MART_NAME):
    os.remove(DATA_MART_NAME)

In [5]:
'''
    This class provides one common point of interaction with my team's database.
    Everything that writes to the database uses this API.
'''
class db_options(Enum):
        DEFAULT = 0
        RETURN_RESULTS = 1
        PRINT_RESULTS = 2

class db:
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql)

    def execute_sql_values(self, sql, values):
        self.cur.execute(sql, values)

    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

#

## 4. Build the product tables

In [6]:
def build_product_table():
    pass